# 5.5.2 Discontinuous Galerkin for the Wave Equation

We solve the first order wave equation by a matrix-free explicit DG method:

\begin{eqnarray*}
\frac{\partial p}{\partial t} & = & \operatorname{div} u \\
\frac{\partial u}{\partial t} & = & \nabla p
\end{eqnarray*}


Using DG discretization in space we obtain the ODE system
\begin{eqnarray*}
M_p \dot{p} & = & -B^T u \\
M_u \dot{u} & = & B p,
\end{eqnarray*}

with mass-matrices $M_p$ and $M_u$, and the discretization matrix $B$ for the gradient, and $-B^T$ for the divergence. 


The DG bilinear-form for the gradient is:

$$
b(p,v) = \sum_{T}
\Big\{ \int_T \nabla p  \, v + \int_{\partial T} (\{ p \} - p) \, v_n \, ds \Big\},
$$
where $\{ p \}$ is the average of $p$ on the facet.

The computational advantages of a proper version of DG is:

* universal element-matrices for the differntial operator $B$
* cheaply invertible mass matrices (orthogonal polynomials, sum-factorization)


In [1]:
from ngsolve import *
from netgen.occ import *
from time import time

from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

box = Box((-1,-1,-1), (1,1,0))
sp = Sphere((0.5,0,0), 0.2)
shape = box-sp
geo = OCCGeometry(shape)

h = 0.1
        
mesh = Mesh( geo.GenerateMesh(maxh=h))
mesh.Curve(3)
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [2]:
order = 4
fes_p = L2(mesh, order=order, all_dofs_together=True)
fes_u = VectorL2(mesh, order=order, piola=True)
fes_tr = FacetFESpace(mesh, order=order)

print ("ndof_p = ", fes_p.ndof, "+", fes_tr.ndof, ", ndof_u =", fes_u.ndof)

traceop = fes_p.TraceOperator(fes_tr, average=True) 

gfu = GridFunction(fes_u)
gfp = GridFunction(fes_p)
gftr = GridFunction(fes_tr)

gfp.Interpolate( exp(-400*(x**2+y**2+z**2)))
gftr.vec.data = traceop * gfp.vec

ndof_p =  574175 + 519660 , ndof_u = 1722525


In [3]:
p = fes_p.TrialFunction()
v = fes_u.TestFunction()
phat = fes_tr.TrialFunction()

n = specialcf.normal(mesh.dim)

$$
b(p,v) = \sum_{T}
\Big\{ \int_T \nabla p  \, v + \int_{\partial T} (\{ p \} - p) \, v_n \, ds \Big\},
$$

where $\{ p \}$ is the average of $p$ on the facet.


In [4]:
Bel = BilinearForm(trialspace=fes_p, testspace=fes_u, geom_free = True)
Bel += grad(p)*v * dx -p*(v*n) * dx(element_boundary=True)
Bel.Assemble()

Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u, geom_free = True)
Btr += phat * (v*n) *dx(element_boundary=True)
Btr.Assemble();

B = Bel.mat + Btr.mat @ traceop

In [5]:
invmassp = fes_p.Mass(1).Inverse()
invmassu = fes_u.Mass(1).Inverse()

In [6]:
gfp.Interpolate( exp(-100*(x**2+y**2+z**2)))
gfu.vec[:] = 0

scene = Draw (gftr, draw_vol=False, order=3, min=-0.05, max=0.05);

t = 0
dt = 0.3 * h / (order+1)**2 
tend = 1

op1 = dt * invmassu @ B
op2 = dt * invmassp @ B.T

cnt = 0
with TaskManager(): 
    while t < tend:
        t = t+dt
        gfu.vec.data += op1 * gfp.vec
        gfp.vec.data -= op2 * gfu.vec
        cnt = cnt+1
        if cnt%10 == 0:
            gftr.vec.data = traceop * gfp.vec
            scene.Redraw()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

## Time-stepping on the device

In [7]:
try:
    import ngsolve
    import ngsolve.ngscuda
    print("NGSolve:", ngsolve.__file__)
    print("ngscuda:", ngsolve.ngscuda.__file__)
except ImportError:
    print("Sorry, no CUDA device or ngscuda not found")

NGSolve:CUDA Device Query...
There is 1 CUDA device.
CUDA Device 0: NVIDIA A100-PCIE-40GB, cap 8.0
Using device 0
 /opt/conda/lib/python3.10/site-packages/ngsolve/__init__.py
ngscuda: /opt/conda/lib/python3.10/site-packages/ngsolve/ngscuda.so
Initializing cublas and cusparse.


In [8]:
gfp.Interpolate(exp(-100*(x**2+y**2+z**2)))
gfu.vec[:] = 0

# re-create the scene right before the GPU loop
scene = Draw(gftr, draw_vol=False, order=3)

t = 0
dt = 0.3 * h / (order+1)**2 
tend = 1

op1 = (dt * invmassu @ B).CreateDeviceMatrix()
op2 = (dt * invmassp @ B.T).CreateDeviceMatrix()

devu = gfu.vec.CreateDeviceVector(copy=True)
devp = gfp.vec.CreateDeviceVector(copy=True)

cnt = 0
with TaskManager():
    while t < tend:
        t += dt
        devu += op1 * devp
        devp -= op2 * devu
        cnt += 1

        # redraw every step first; later bump this back to 10
        if cnt % 10 == 0:
            # (optional) if you suspect async issues, uncomment next line:
            # ngsolve.CudaSynchronize()
            gfp.vec.data = devp                 # device -> host copy
            gftr.vec.data = traceop * gfp.vec   # update trace from host vec
            scene.Redraw()
            # quick heartbeat so you know it's running:
            print("step", cnt)



WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

step 10
step 20
step 30
step 40
step 50
step 60
step 70
step 80
step 90
step 100
step 110
step 120
step 130
step 140
step 150
step 160
step 170
step 180
step 190
step 200
step 210
step 220
step 230
step 240
step 250
step 260
step 270
step 280
step 290
step 300
step 310
step 320
step 330
step 340
step 350
step 360
step 370
step 380
step 390
step 400
step 410
step 420
step 430
step 440
step 450
step 460
step 470
step 480
step 490
step 500
step 510
step 520
step 530
step 540
step 550
step 560
step 570
step 580
step 590
step 600
step 610
step 620
step 630
step 640
step 650
step 660
step 670
step 680
step 690
step 700
step 710
step 720
step 730
step 740
step 750
step 760
step 770
step 780
step 790
step 800
step 810
step 820
step 830


In [9]:
gfp.Interpolate( exp(-100*(x**2+y**2+z**2)))
gfu.vec[:] = 0

scene = Draw (gftr, draw_vol=False, order=3);

t = 0
dt = 0.5 * h / (order+1)**2 / 2
tend = 0.1

op1 = (dt * invmassu @ B).CreateDeviceMatrix()
op2 = (dt * invmassp @ B.T).CreateDeviceMatrix()

devu = gfu.vec.CreateDeviceVector(copy=True)
devp = gfp.vec.CreateDeviceVector(copy=True)

cnt = 0
with TaskManager(): 
    while t < tend:
        t = t+dt
        devu += op1 * devp
        devp -= op2 * devu
        cnt = cnt+1
        if cnt%10 == 0:
            gfp.vec.data = devp
            gftr.vec.data = traceop * gfp.vec
            scene.Redraw()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [10]:
print (op1.GetOperatorInfo())

ProductMatrix, h = 1722525, w = 574175
  ScaleMatrix, scale = 0.001, h = 1722525, w = 1722525
    SumMatrix, h = 1722525, w = 1722525
      SumMatrix, h = 1722525, w = 1722525
        N4ngla25DevBlockDiagonalMatrixSoAE, h = 1722525, w = 1722525
        ProductMatrix, h = 1722525, w = 1722525
          ProductMatrix, h = 1722525, w = 19875
            Transpose, h = 1722525, w = 19875
              N4ngla33DevConstantElementByElementMatrixE, h = 19875, w = 1722525
            N4ngla25DevBlockDiagonalMatrixSoAE, h = 19875, w = 19875
          N4ngla33DevConstantElementByElementMatrixE, h = 19875, w = 1722525
      ProductMatrix, h = 1722525, w = 1722525
        ProductMatrix, h = 1722525, w = 21750
          Transpose, h = 1722525, w = 21750
            N4ngla33DevConstantElementByElementMatrixE, h = 21750, w = 1722525
          N4ngla25DevBlockDiagonalMatrixSoAE, h = 21750, w = 21750
        N4ngla33DevConstantElementByElementMatrixE, h = 21750, w = 1722525
  SumMatrix, h = 1722525, w =

In [11]:
ts = time()
steps = 10
for i in range(steps):
    devu += op1 * devp
    devp -= op2 * devu
te = time()
print ("ndof = ", gfp.space.ndof, "+", gfu.space.ndof, ", time per step =", (te-ts)/steps)

ndof =  574175 + 1722525 , time per step = 0.0014894485473632812


On the A-100:

## Time-domain PML

PML (perfectly matched layers) is a method for approximating outgoing waves on a truncated domain. Its time domain version leads to additional field variables in the PML region, which are coupled via time-derivatives only.

In [12]:
from ring_resonator_import import *
from ngsolve import *
from ngsolve.webgui import Draw

ModuleNotFoundError: No module named 'ring_resonator_import'

In [ ]:
gfu = GridFunction(fes)
scene = Draw (gfu.components[0], order=3, min=-0.05, max=0.05, autoscale=False)

t = 0
tend = 15
tau = 2e-4
i = 0
sigma = 10   # pml damping parameter

op1 = invp@(-fullB.T-sigma*dampingp) 
op2 = invu@(fullB-sigma*dampingu)
with TaskManager(): 
    while t < tend:

        gfu.vec.data += tau*Envelope(t)*srcvec
        gfu.vec.data += tau*op1*gfu.vec
        gfu.vec.data += tau*op2*gfu.vec        

        t += tau
        i += 1
        if i%20 == 0:
            scene.Redraw()

The differential operators and coupling terms to the pml - variables are represented via linear operators;

In [ ]:
print (fullB.GetOperatorInfo())
print (dampingp.GetOperatorInfo())

In [ ]:
print (op1.GetOperatorInfo())